In [58]:
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

In [59]:
def generate_system(n):
  A = np.random.rand(n, n)
  A = np.matrix(0.5 * (A + A.T) + n * np.identity(n))
  return A, np.matrix(np.random.rand(n, 1))

$P = \tau^{-1}I$

$P(x^{k+1} - x^k) = \tau^k(b - Ax^k)$

$x^{k+1} = \tau^kP^{-1}(b - Ax^k) + x^k$

In [65]:
class RandomConstStepChooser:
  step = None

  def __init__(self, A):
    eigs, _ = np.linalg.eig(A)
    self.step = np.random.rand(1)[0] * 2 / eigs.max()

  def next_step(self, A, P_inv, b, x):
    return self.step

class OptConstStepChooser:
  step = None

  def __init__(self, A):
    eigs, _ = np.linalg.eig(A)
    self.step = 2 / (abs(eigs.min()) + abs(eigs.max()))

  def next_step(self, A, P_inv, b, x):
    return self.step

class AlmostOptConstStepChooser:
  step = None

  def __init__(self, A):
    n = A.shape[0]
    max = A.max() * n * n
    min = A.min() * n * n
    for i in range(n):
      r = 0
      if i > 0:
        r += np.abs(A[i,:i-1]).sum()
      if i < n - 1:
        r += np.abs(A[i,i+1:]).sum()
      min_lamb = A[i,i] - r
      max_lamb = A[i,i] + r
      if max < max_lamb:
        max = max_lamb
      if min > min_lamb:
        min = min_lamb
    abs_max = max(abs(max), abs(min))
    abs_min = min(abs(max), abs(max))
    if max * min < 0:
      abs_min = 0
    self.step = 2 / (abs_min + abs_max)

  def next_step(self, A, P_inv, b, x):
    return self.step

class DynamicStepChooser:
  def __init__(self, A):
    pass

  def next_step(self, A, P_inv, b, x):
    r = b - A * x
    z = P_inv * r
    return (z.T * r)[0,0] / (z.T * A * z)[0,0]

In [69]:
def solve(A, b, stepChooser, P=None, n_iter=1000):
  assert A.shape[0] == A.shape[1]
  assert A.shape[0] == b.shape[0]
  assert (A == A.T).all()
  n = A.shape[0]
  sol_norm = np.linalg.norm(np.linalg.solve(A, b))
  if P is None:
    P = np.matrix(np.identity(n))
  P_inv = np.linalg.inv(P)
  P_inv_A = P_inv * A
  x =  np.matrix(np.ones((n, 1)))
  errs = []
  for _ in range(n_iter):
    err = b - A * x
    errs += [np.linalg.norm(err) / sol_norm]
    x += stepChooser.next_step(A, P_inv, b, x) * P_inv * err
  return x

In [71]:
A, b = generate_system(500)
step_chooser = DynamicStepChooser(A)
x = solve(A, b, step_chooser)
np.linalg.norm(A * x - b) / np.linalg.norm(np.linalg.solve(A, b))

3.829259632817029e-14